In [65]:
import os
import numpy as np
import tensorflow as tf

def load_names(namefile):
    """Loads DeepSEA label names from namefile.
    """
    return np.array(open(namefile).read().split(","))


def get_overlapping_indices_for_cell_type(cell_type_1, cell_type_2):
    """Gets target indices for transcription factors for the intersection 
    of cell_type_1 and cell_type_2.

    Args:
      cell_type_1: Name of cell type 1 as a string.
      cell_type_2: Name of cell type 2 as a string.

    Returns:
      List of indices ~ FOR CELL 1 ~ that overlap with cell 2.
      These indices are listed in alphabetical order for consistency.
    """
    
#     dir_path = os.path.dirname(os.path.realpath(__file__))
    namefile = "../tfti/deepsea_label_names.txt"
    names = load_names(namefile)
    
    valid_cell_types = list(map(lambda x: x.split("|")[1], names))
    
    # Make sure cell type parameters can be found in our data. 
    assert(cell_type_1 in valid_cell_types, "{} not in list of valid cell types".format(cell_type_1))
    assert(cell_type_2 in valid_cell_types, "{} not in list of valid cell types".format(cell_type_2))
        
    # Get positions for both cell lines.
    cell_type_1_pos = [(i, j) for i, j in enumerate(names) \
                        if cell_type_1 in j]
    cell_type_2_pos = [(i, j) for i, j in enumerate(names) \
                        if cell_type_2 in j]

    # Get marks for each cell type
    cell_type_1_marks =  [i[1].split("|")[1] for i in cell_type_1_pos]
    cell_type_2_marks =  [i[1].split("|")[1] for i in cell_type_2_pos]
    
    # Get overlapping marks between both cell types.
    overlapping_marks = list(set(cell_type_1_marks) & set(cell_type_2_marks))

    cell_type_1_final_pos = [(i,j) for i, j in cell_type_1_pos if \
                                    j.split("|")[1] in overlapping_marks]
    cell_type_2_final_pos = [(i,j) for i, j in  cell_type_2_pos if \
                                    j.split("|")[1] in overlapping_marks]

    # Filter out duplicates for both cell types.
    cell_type_1_items = []
    seen = set()
    for item in cell_type_1_final_pos:
      if item[1] not in seen:
        seen.add(item[1])
        cell_type_1_items.append(item)

    cell_type_1_items = sorted(cell_type_1_items, key=lambda i: i[1]) 
    
    # Print out sorted marks.
    tf.logging.info("Marks for CellType %s: %s" 
                    % (cell_type_1, 
                    cell_type_1_items))

    cell_type_2_items = []
    seen = set()
    for item in cell_type_2_final_pos:
      if not item[1] in seen:
        seen.add(item[1])
        cell_type_2_items.append(item)

    cell_type_2_items = sorted(cell_type_2_items, key=lambda i: i[1])

    # Verify that TFs match between cell types.
    for i, item in enumerate(cell_type_2_items):
      assert(cell_type_2_items[i][1].split("|")[1] ==
             cell_type_1_items[i][1].split("|")[1])

    # These are the indices we are using for the cell type 1 model.
    cell_type_1_indices = list(map(lambda x: x[0], cell_type_1_items))
    return cell_type_1_indices

<ipython-input-65-96330ca81c95>:31: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(cell_type_1 in valid_cell_types, "{} not in list of valid cell types".format(cell_type_1))
<ipython-input-65-96330ca81c95>:32: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(cell_type_2 in valid_cell_types, "{} not in list of valid cell types".format(cell_type_2))


In [66]:
namefile = "../tfti/deepsea_label_names.txt"
example = load_names(namefile)

gather_indices = get_overlapping_indices_for_cell_type("GM12878", "H1-hESC")
example[gather_indices]

array(['GM12878|ATF2|None', 'GM12878|ATF3|None', 'GM12878|BCL11A|None',
       'GM12878|BRCA1|None', 'GM12878|CEBPB|None', 'GM12878|CHD1|None',
       'GM12878|CHD2|None', 'GM12878|CTCF|None', 'GM12878|DNase|None',
       'GM12878|EZH2|None', 'GM12878|Egr-1|None', 'GM12878|GABP|None',
       'GM12878|JunD|None', 'GM12878|Max|None', 'GM12878|Mxi1|None',
       'GM12878|NRSF|None', 'GM12878|Nrf1|None', 'GM12878|Pol2-4H8|None',
       'GM12878|Pol2|None', 'GM12878|RFX5|None', 'GM12878|RXRA|None',
       'GM12878|Rad21|None', 'GM12878|SIN3A|None', 'GM12878|SIX5|None',
       'GM12878|SP1|None', 'GM12878|SRF|None', 'GM12878|TAF1|None',
       'GM12878|TBP|None', 'GM12878|TCF12|None', 'GM12878|USF-1|None',
       'GM12878|USF2|None', 'GM12878|YY1|None', 'GM12878|Znf143|None',
       'GM12878|c-Myc|None', 'GM12878|p300|None'], dtype='<U37')

In [52]:
argsort_indices = np.argsort(gather_indices)
gather_indices_sorted = np.sort(gather_indices)

# Keep targets and latents corresponding to GM12878 (LCL cell line).
targets = example[gather_indices_sorted]
latents = example[gather_indices_sorted]
targets_copy = np.array(list(targets))

targets

array(['H1-hESC|DNase|None', 'H1-hESC|CHD1|None', 'H1-hESC|CTCF|None',
       'H1-hESC|EZH2|None', 'H1-hESC|ATF2|None', 'H1-hESC|ATF3|None',
       'H1-hESC|BCL11A|None', 'H1-hESC|Egr-1|None', 'H1-hESC|GABP|None',
       'H1-hESC|JunD|None', 'H1-hESC|NRSF|None', 'H1-hESC|p300|None',
       'H1-hESC|Pol2-4H8|None', 'H1-hESC|Pol2|None', 'H1-hESC|Rad21|None',
       'H1-hESC|RXRA|None', 'H1-hESC|SIX5|None', 'H1-hESC|SP1|None',
       'H1-hESC|SRF|None', 'H1-hESC|TAF1|None', 'H1-hESC|TCF12|None',
       'H1-hESC|USF-1|None', 'H1-hESC|YY1|None', 'H1-hESC|BRCA1|None',
       'H1-hESC|CEBPB|None', 'H1-hESC|CHD2|None', 'H1-hESC|c-Myc|None',
       'H1-hESC|Max|None', 'H1-hESC|Mxi1|None', 'H1-hESC|Nrf1|None',
       'H1-hESC|RFX5|None', 'H1-hESC|SIN3A|None', 'H1-hESC|TBP|None',
       'H1-hESC|USF2|None', 'H1-hESC|Znf143|None'], dtype='<U37')

In [69]:
list(list(argsort_indices))

[8,
 5,
 7,
 9,
 0,
 1,
 2,
 10,
 11,
 12,
 15,
 34,
 17,
 18,
 21,
 20,
 23,
 24,
 25,
 26,
 28,
 29,
 31,
 3,
 4,
 6,
 33,
 13,
 14,
 16,
 19,
 22,
 27,
 30,
 32]

In [70]:
# This is how we are currently realphabetizing. 

# Ensure sure tensors are sorted by alphabetical TFs.
targets[argsort_indices]


array(['H1-hESC|DNase|None', 'H1-hESC|CHD1|None', 'H1-hESC|CTCF|None',
       'H1-hESC|EZH2|None', 'H1-hESC|ATF2|None', 'H1-hESC|ATF3|None',
       'H1-hESC|BCL11A|None', 'H1-hESC|Egr-1|None', 'H1-hESC|GABP|None',
       'H1-hESC|JunD|None', 'H1-hESC|NRSF|None', 'H1-hESC|p300|None',
       'H1-hESC|Pol2-4H8|None', 'H1-hESC|Pol2|None', 'H1-hESC|Rad21|None',
       'H1-hESC|RXRA|None', 'H1-hESC|SIX5|None', 'H1-hESC|SP1|None',
       'H1-hESC|SRF|None', 'H1-hESC|TAF1|None', 'H1-hESC|TCF12|None',
       'H1-hESC|USF-1|None', 'H1-hESC|YY1|None', 'H1-hESC|BRCA1|None',
       'H1-hESC|CEBPB|None', 'H1-hESC|CHD2|None', 'H1-hESC|c-Myc|None',
       'H1-hESC|Max|None', 'H1-hESC|Mxi1|None', 'H1-hESC|Nrf1|None',
       'H1-hESC|RFX5|None', 'H1-hESC|SIN3A|None', 'H1-hESC|TBP|None',
       'H1-hESC|USF2|None', 'H1-hESC|Znf143|None'], dtype='<U37')

In [54]:
# This is how we should be doing it.

inds = list(range(len(targets)))
targets_copy[argsort_indices] = targets[inds]
targets_copy


array(['H1-hESC|ATF2|None', 'H1-hESC|ATF3|None', 'H1-hESC|BCL11A|None',
       'H1-hESC|BRCA1|None', 'H1-hESC|CEBPB|None', 'H1-hESC|CHD1|None',
       'H1-hESC|CHD2|None', 'H1-hESC|CTCF|None', 'H1-hESC|DNase|None',
       'H1-hESC|EZH2|None', 'H1-hESC|Egr-1|None', 'H1-hESC|GABP|None',
       'H1-hESC|JunD|None', 'H1-hESC|Max|None', 'H1-hESC|Mxi1|None',
       'H1-hESC|NRSF|None', 'H1-hESC|Nrf1|None', 'H1-hESC|Pol2-4H8|None',
       'H1-hESC|Pol2|None', 'H1-hESC|RFX5|None', 'H1-hESC|RXRA|None',
       'H1-hESC|Rad21|None', 'H1-hESC|SIN3A|None', 'H1-hESC|SIX5|None',
       'H1-hESC|SP1|None', 'H1-hESC|SRF|None', 'H1-hESC|TAF1|None',
       'H1-hESC|TBP|None', 'H1-hESC|TCF12|None', 'H1-hESC|USF-1|None',
       'H1-hESC|USF2|None', 'H1-hESC|YY1|None', 'H1-hESC|Znf143|None',
       'H1-hESC|c-Myc|None', 'H1-hESC|p300|None'], dtype='<U21')

In [71]:
# Get GM12878 order.

act = list(targets[argsort_indices])
exp = list(targets_copy)
gm12878_order = [exp.index(val) for val in act]
print(gm12878_order)

[8, 5, 7, 9, 0, 1, 2, 10, 11, 12, 15, 34, 17, 18, 21, 20, 23, 24, 25, 26, 28, 29, 31, 3, 4, 6, 33, 13, 14, 16, 19, 22, 27, 30, 32]


In [72]:
# Order H1-hESC labels the same way as GM12878.

gather_indices = get_overlapping_indices_for_cell_type("H1-hESC", "GM12878")

# Keep targets and latents corresponding to GM12878 (LCL cell line).
targets = example[gather_indices]

targets[gm12878_order]

array(['H1-hESC|DNase|None', 'H1-hESC|CHD1|None', 'H1-hESC|CTCF|None',
       'H1-hESC|EZH2|None', 'H1-hESC|ATF2|None', 'H1-hESC|ATF3|None',
       'H1-hESC|BCL11A|None', 'H1-hESC|Egr-1|None', 'H1-hESC|GABP|None',
       'H1-hESC|JunD|None', 'H1-hESC|NRSF|None', 'H1-hESC|p300|None',
       'H1-hESC|Pol2-4H8|None', 'H1-hESC|Pol2|None', 'H1-hESC|Rad21|None',
       'H1-hESC|RXRA|None', 'H1-hESC|SIX5|None', 'H1-hESC|SP1|None',
       'H1-hESC|SRF|None', 'H1-hESC|TAF1|None', 'H1-hESC|TCF12|None',
       'H1-hESC|USF-1|None', 'H1-hESC|YY1|None', 'H1-hESC|BRCA1|None',
       'H1-hESC|CEBPB|None', 'H1-hESC|CHD2|None', 'H1-hESC|c-Myc|None',
       'H1-hESC|Max|None', 'H1-hESC|Mxi1|None', 'H1-hESC|Nrf1|None',
       'H1-hESC|RFX5|None', 'H1-hESC|SIN3A|None', 'H1-hESC|TBP|None',
       'H1-hESC|USF2|None', 'H1-hESC|Znf143|None'], dtype='<U37')